### 0 - Configuration

In [ ]:
!pip install sagemaker python-dotenv --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.7/844.7 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.9 MB/s eta 0:00:00


In [ ]:
import sagemaker
import boto3

import os
from dotenv import load_dotenv

### 1 - AWS Session

In [ ]:
# environment variables
# Option 1
# os.environ["aws_access_key_id"]='aws_access_key_id'
# os.environ["aws_secret_access_key"]='aws_secret_access_key'

# Option 2
load_dotenv()

True

  [Secret keys](https://docs.aws.amazon.com/powershell/latest/userguide/pstools-appendix-sign-up.html)

In [ ]:
REGION_NAME = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME
ROLE_NAME =  'Sagemaker-ExecutionRole'

auth_arguments = {
    'aws_access_key_id':os.environ["aws_access_key_id"],
    'aws_secret_access_key':os.environ["aws_secret_access_key"],
    'region_name':REGION_NAME
}

[IAM rol](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)

In [ ]:
iam = boto3.client('iam', **auth_arguments)
role = iam.get_role(RoleName=ROLE_NAME)['Role']['Arn']

session = sagemaker.Session(boto3.Session(**auth_arguments))

### 2 - Deployment

In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# image uri
llm_image = get_huggingface_llm_image_uri("huggingface")

print(f"image uri: {llm_image}")

image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04


In [ ]:
from sagemaker.huggingface import HuggingFaceModel

# Falcon 7b
hub = {'HF_MODEL_ID':'tiiuae/falcon-7b'}

# Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role,  # iam role from AWS
   image_uri=llm_image,
   sagemaker_session=session
)

In [ ]:
# deploy model to SageMaker
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.g5.2xlarge', #'ml.g5.4xlarge'
 	container_startup_health_check_timeout=300
)

--------------!

### 3 - Inference

In [ ]:
# Placeholder for conversation history; in a real application, this would be more dynamically managed
conversation_history = [{"role": "system", "content": "You are a helpful assistant."}]

def generate_response(user_input):
    """
    Generate a response from the model based on the user's input and the conversation history.
    """
    global conversation_history
    # Add the latest user input to the conversation history
    conversation_history.append({"role": "user", "content": user_input})
    
    # Prepare the prompt by including the conversation history
    prompt_with_history = "\n".join(conversation_history) + "\nassistant:"
    
    # Hyperparameters for the LLM request, with the updated prompt
    request = {
        "inputs": prompt_with_history,
        "parameters": {
            "do_sample": True,
            "top_p": 0.9,
            "temperature": 0.7,
            "max_new_tokens": 512,
            "stop": ["\nUser:","","</s>"]
        }
    }
    
    # Request to the endpoint
    response = predictor.predict(request)
    
    # Extracting model response
    model_response = response[0]["generated_text"].split("assistant:")[-1].strip()
    
    # Add model response to the conversation history
    conversation_history.append({"role": "assistant", "content": model_response})
    
    # Return the model's response
    return model_response

# Example usage
user_input = "What is the capital of Spain?"
assistant_response = generate_response(user_input)

In [ ]:
print(assistant_response)

 Madrid.
User:


In [ ]:
# DELETE ENDPOINT to avoid unnecessary expenses
predictor.delete_model()
predictor.delete_endpoint()